In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("US_Accidents_Dec21_updated.csv")
df.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.108910,-83.092860,40.112060,-83.031870,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.865420,-84.062800,39.865010,-84.048730,0.747,At OH-4/OH-235/Exit 41 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-3,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.102660,-84.524680,39.102090,-84.523960,0.055,At I-71/US-50/Exit 1 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Day
3,A-4,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.062130,-81.537840,41.062170,-81.535470,0.123,At Dart Ave/Exit 21 - Accident.,...,False,False,False,False,False,False,Night,Night,Day,Day
4,A-5,3,2016-02-08 07:53:43,2016-02-08 13:53:43,39.172393,-84.492792,39.170476,-84.501798,0.500,At Mitchell Ave/Exit 6 - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day


## Creating the Location Dimension dataframe and updating the original dataframe:

In [3]:
Location_ID = df.groupby(['City', 'County', 'State', 'Zipcode', 'Country', 'Timezone'], dropna=False).ngroup()

In [4]:
insert_loc = df.columns.get_loc("Side") + 1

In [5]:
df.insert(loc = insert_loc, column = "Location_ID", value = Location_ID)

In [6]:
df_location = df.loc[:, "Location_ID": "Timezone"]

In [7]:
df_location = df_location.drop_duplicates()

In [8]:
#df_location[df_location['Zipcode'].isnull()]

In [9]:
df = df.drop(['City', 'County', 'State', 'Zipcode', 'Country', 'Timezone'], axis = 1)

## Creating the POI Dimension dataframe and updating the original dataframe:

In [10]:
POI_ID = df.groupby(['Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop'], dropna = False).ngroup()

In [11]:
insert_loc = df.columns.get_loc("Weather_Condition") + 1

In [12]:
df.insert(loc = insert_loc, column = "POI_ID", value = POI_ID)

In [13]:
df_POI = df.loc[:, "POI_ID": "Turning_Loop"]

In [14]:
df_POI = df_POI.drop_duplicates()

In [15]:
df = df.drop(['Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop'], axis = 1)

## Renaming the columns to work in BigQuery and transforming some columns 

In [16]:
df.columns = ['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'End_Lat', 'End_Lng', 'Distance_mi', 'Description', 'Number', 'Street',
       'Side', 'Location_ID', 'Airport_Code', 'Weather_Timestamp',
       'Temperature_F', 'Wind_Chill_F', 'Humidity', 'Pressure_in',
       'Visibility_mi', 'Wind_Direction', 'Wind_Speed_mph',
       'Precipitation_in', 'Weather_Condition', 'POI_ID', 'Sunrise_Sunset',
       'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']

In [17]:
df['Start_Time'] = pd.to_datetime(df['Start_Time']).dt.floor('S')
df['End_Time'] = pd.to_datetime(df['End_Time']).dt.floor('S')
df['Weather_Timestamp'] = pd.to_datetime(df['Weather_Timestamp']).dt.floor('S')

## Exporting the 3 dataframes into separate CSV files 

In [18]:
df.to_csv("Accidents_fact_table.csv", index = False)

In [19]:
df_location.to_csv("Location_dimension_table.csv", index = False)

In [20]:
df_POI.to_csv("POI_dimension_table.csv", index = False)